# ST2MLE : Machine Learning for IT Engineers Project
## Machine Learning Project – Numerical and Textual Data (French Context)
### Context
As part of this project, students will work on mixed data (numerical and textual) collected from French websites.
The objective is to carry out a comprehensive analysis, from data collection to modeling
and interpretation, with a focus on a French economic, social, or public context.

### Learning Objectives
- Master the full lifecycle of a data project (collection, cleaning, preprocessing, modeling, evaluation).
- Apply techniques for text processing and numerical data analysis.
- Explore various text vectorization techniques (BoW, TF-IDF, Doc2Vec, BERT).
- Conduct analyses and provide recommendations based on real French data.

### Project Steps
1. Define a topic, the needs and identify relevant French sources.
2. Collect data (web scraping, APIs...).
3. Clean and preprocess both numerical and textual data.
4. Annotate (label) data. Some websites already include categories or tags — these can be scraped alongside the text and used as labels. Otherwise, label data manually.
5. Perform exploratory analysis and visualizations (distributions, word clouds, correlations...) to check for outliers, class imbalance, etc.
6. Apply under-sampling or oversampling (if needed), PCA for feature extraction (if needed).
7. Apply predictive models:
  - **Numerical data**: Decision Trees, Random Forest, Boosting.
  - **Textual data**: Naive Bayes, Logistic Regression after vectorization.
8. Compare text vectorization methods: BoW, TF-IDF, Doc2Vec, BERT.
9. Provide business recommendations and submit a final report.

### Technical Constraints
- Data must be exclusively from French sources.
- Texts must be in French only (use appropriate preprocessing: French lemmatization, French stopwords).
- Minimum of 500 data rows.
- Implementation in Python using scikit-learn, gensim, transformers, etc.

### Evaluation Criteria
- Quality and relevance of data collection and labeling (10%)
- Quality of data cleaning and preprocessing (10%)
- Relevance of visualizations and exploratory analysis (10%)
- Implementation of models (30%)
- Comparison and discussion of vectorization techniques (10%)
- Recommendations and critical thinking (10%)
- Quality of the report and code (5%)
- Quality of the presentation (5%)
- Q&A (10%)

In [ ]:
import numpy as np

# Import data from CSV file

import pandas as pd

# Load the CSV file
df = pd.read_csv('data/classements_letudiant.csv')

# Display information of dataset
df.info()

df

In [ ]:
# Update column name to snake_case with no accents and no /10 (max score suffix)
df.columns = (
    df.columns.str.lower()
    .str.strip()
    .str.replace(" /10", "")
    .str.replace(" ", "_")
    .str.replace("é", "e")
    .str.replace("è", "e")
)

df

In [ ]:
# Remove /10 /2 (max score suffix) in scores
# Remove % in scores
df["score"] = (
    df["score"]
    .astype(str)
    .str.replace(r"/\d+", "", regex=True)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

df

In [ ]:
# Pivot table
df = df.pivot_table(
    index="ecole", columns="critere", values=["score", "note_brute"], aggfunc="first"
).sort_index()

df

In [ ]:
# Rename columns to snake_case
df.columns = [f"{v}_{c}".lower().replace(" ", "_") for v, c in df.columns]

df.reset_index(inplace=True)

df

In [ ]:
# Delete specific columns
columns_to_drop = [
    "note_brute_origine_des_intégrés_en_cycle_ingénieur",
    "note_brute_nombre_d\'intégrés_issus_de_bac_technologique",
    "note_brute_nombre_d\'intégrés_issus_de_bac_général",
]  # All have "Voir plus" value - Useless for analysis

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Go in every note_brute column
# Replace "Non communiqué" with None
# Replace "Oui" by True and "Non" by False

excluded_columns = [
    "note_brute_concours",
    "note_brute_label_dd&rs",
    "note_brute_ministère_de_tutelle",
    "note_brute_masse_et_encadrement_des_doctorants",
    "note_brute_niveau_d\'anglais_exigé",
    "note_brute_ouverture_sociale",
]

for col in df.columns:
    if "note_brute" in col and col not in excluded_columns:
        df[col] = (
            df[col]
            .replace("%", "", regex=True)
            .replace("Non communiqué", None)
            .replace("Oui", True)
            .replace("Non", False)
            .replace({pd.NA: np.nan, None: np.nan})
        )

        df[col] = df[col].apply(
            lambda x: (
                float(str(x).replace(",", "."))
                if isinstance(x, str) and str(x).replace(",", ".").replace(".", "", 1).isdigit()
                else x
            )
        )

        if df[col].dtype == "object":
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass  # Ignore columns that cannot be converted to float (like False/ T/F/str

# Label DD&RS -> True/False
df["note_brute_label_dd&rs"] = df["note_brute_label_dd&rs"].replace(
    {"Label DD&RS": True, "Pas de label": False}
)

# note_brute_masse_et_encadrement_des_doctorants & note_brute_ouverture_sociale -> Numerical values
level_mapping = {
    "Aucun": 0,
    "Faible": 1,
    "Correcte": 2,
    "Moyenne": 3,
    "Importante": 4,
    "Très importante": 5,
}

df["note_brute_masse_et_encadrement_des_doctorants"] = df[
    "note_brute_masse_et_encadrement_des_doctorants"
].map(level_mapping)

level_mapping = {"Faible": 1, "Passable": 2, "Correcte": 3, "Bonne": 4, "Excellente": 5}

df["note_brute_ouverture_sociale"] = df["note_brute_ouverture_sociale"].map(level_mapping)

# Parité homme femmes
df["note_brute_parité_au_sein_de_la_promotion_(hommes/femmes)"] = (
    df["note_brute_parité_au_sein_de_la_promotion_(hommes/femmes)"]
    .astype(str)
    .str.replace(",", ".", regex=False)
)
df[["note_brute_part_etudiant_hommes", "note_brute_part_etudiant_femmes"]] = (
    df["note_brute_parité_au_sein_de_la_promotion_(hommes/femmes)"]
    .str.split("|", expand=True)
    .apply(pd.to_numeric, errors="ignore")
)

# Drop the original column
df.drop(
    columns=["note_brute_parité_au_sein_de_la_promotion_(hommes/femmes)"],
    inplace=True,
    errors="ignore",
)

df

In [ ]:
def export_df_to_csv(df, filename):
    """
    Export DataFrame to CSV file.

    Parameters:
    df (DataFrame): The DataFrame to export.
    filename (str): The name of the output CSV file.
    """
    df.to_csv(filename, index=False)


# export_df_to_csv(df, 'numerical_values.csv')

In [ ]:
df.info()

In [ ]:
# EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === Corrélation (heatmap) ===

# Sélection des colonnes numériques uniquement
numeric_df = df.select_dtypes(include='number')

# Heatmap des corrélations sans annotations
plt.figure(figsize=(12, 10))
sns.heatmap(numeric_df.corr(), annot=False, fmt=".2f", cmap='coolwarm', square=True,
            xticklabels=False, yticklabels=False)
plt.axis('off')
plt.tight_layout()
plt.show()

# === Boxplot de l'insertion à 2 mois ===

plt.figure(figsize=(10, 6))
sns.boxplot(data=df, y='note_brute_insertion_à_deux_mois')
plt.title('Répartition des taux d\'insertion à 2 mois')
plt.ylabel('Taux d\'insertion (%)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# === Radar Chart pour international ===

# Liste des critères sélectionnés
criteria = [
    "note_brute_pourcentage_d'étudiants_internationaux",
    "note_brute_diplômés_en_poste_à_l'international",
    "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_stage"
]

# Convertir en numérique et gérer les erreurs sur tout le df (pour être sûr)
for col in criteria:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Remplacer NaN par la moyenne de la colonne
df[criteria] = df[criteria].fillna(df[criteria].mean())

# Calcul du score international = moyenne des 3 critères
df['score_international'] = df[criteria].mean(axis=1)

# Sélectionner top 5 écoles selon score international
top5_schools = df.sort_values(by='score_international', ascending=False).head(5)

# Extraire données critères pour ces écoles
data = top5_schools.set_index('ecole')[criteria]

# Normalisation min-max par colonne (critères)
normalized_data = (data - data.min()) / (data.max() - data.min())

# Préparer labels (numéros) et angles pour le radar chart
num_vars = len(criteria)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # Fermer le polygone

labels_num = [str(i) for i in range(1, num_vars + 1)]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

colors = sns.color_palette("RdBu_r", n_colors=len(normalized_data))

# Tracer chaque école avec numéro comme légende et couleurs rouge à bleu
for i, (school, color) in enumerate(zip(normalized_data.index, colors), start=1):
    values = normalized_data.loc[school].tolist()
    values += values[:1]
    ax.plot(angles, values, label=str(i), color=color)
    ax.fill(angles, values, alpha=0.25, color=color)

ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_thetagrids(np.degrees(angles[:-1]), labels_num, fontsize=12)
plt.xticks(rotation=45)
plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], color="grey", size=8)
ax.set_rlabel_position(180 / num_vars)
plt.ylim(0, 1)

plt.title("Top 5 écoles selon score international", size=16, y=1.1)
plt.legend(title="Écoles (numéro)", loc='upper right', bbox_to_anchor=(1.3, 1.1))

# Tableau sous le radar chart (inchangé)

table_data = top5_schools.set_index('ecole')[criteria + ['score_international']].round(2)
table_data_num = table_data[criteria].copy()
table_data_num.columns = labels_num
table_data_num['Score International'] = table_data['score_international']
table_data_num.index.name = 'École'

plt.subplots_adjust(bottom=0.3)
table_ax = fig.add_axes([0.1, 0.05, 0.8, 0.2])
table_ax.axis('off')

table = table_ax.table(cellText=table_data_num.reset_index().values,
                       colLabels=table_data_num.reset_index().columns,
                       cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.show()

print("Correspondance numéro -> critère :")
for i, crit in enumerate(criteria, start=1):
    print(f"{i} : {crit}")

print("\nCorrespondance numéro -> école :")
for i, school in enumerate(normalized_data.index, start=1):
    print(f"{i} : {school}")

# === Radar Chart pour une école (ex : EFREI) ===

features = [
    # Stage & Expérience
    "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_stage",
    "note_brute_durée_minimale_de_stages_en_entreprise",
    "note_brute_taux_d'alternants",
    # Vie Étudiante & Accompagnement
    "note_brute_valorisation_de_l'engagement_étudiant",
    "note_brute_nombre_de_places_en_résidence_crous_et_privées",
    # International
    "note_brute_pourcentage_d'étudiants_internationaux",
    "note_brute_diplômés_en_poste_à_l'international",
    "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_échange_académique",
    "score_diplômés_en_poste_à_l'international",
    # Formation & Effectifs
    "note_brute_cycle_ingénieur_-_nombre_d'intégrés_en_1ère_année",
    "note_brute_cycle_prépa_intégrée_-_nombre_d'intégrés_à_bac",
    "note_brute_nombre_d'étudiants_par_enseignant",
    "note_brute_taille_des_promos_en_cycle_ingénieur",
    "note_brute_part_etudiant_hommes",
    "note_brute_part_etudiant_femmes",
    # Qualité & Recherche
    "note_brute_recherche_et_enseignement",
    "note_brute_masse_et_encadrement_des_doctorants",
    "score_label_dd&rs",
    "score_masse_et_encadrement_des_doctorants"
]

ecole_cible = "EFREI"
row = df[df["ecole"].str.contains(ecole_cible, case=False, na=False)]

if row.empty:
    print(f"L'école {ecole_cible} n'a pas été trouvée.")
else:
    values = row[features].iloc[0].fillna(0).values
    max_vals = df[features].max().values
    values = values / max_vals

    labels = [f.replace("note_brute_", "").replace("_", " ")[:25] for f in features]
    num_vars = len(labels)

    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    values = values.tolist()
    values += values[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))
    ax.plot(angles, values, color='tab:blue', linewidth=2)
    ax.fill(angles, values, color='tab:blue', alpha=0.25)

    ax.set_title(f"Radar Chart - {ecole_cible}", size=16, y=1.08)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels, fontsize=10)
    ax.set_ylim(0, 1)
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# === Dictionnaire des catégories pour les heatmap ===
criteria_dict = {
    "Stage & Expérience": {
        '1': "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_stage",
        '2': "note_brute_durée_minimale_de_stages_en_entreprise",
        '3': "note_brute_pourcentage_d'étudiants_en_stage_en_fin_d'études",
        '4': "note_brute_taux_d'alternants",
    },
    "Vie Étudiante & Accompagnement": {
        '1': "note_brute_valorisation_de_l'engagement_étudiant",
        '2': "note_brute_vie_associative",
        '3': "note_brute_nombre_de_places_en_résidence_crous_et_privées"
    },
    "International": {
        '1': "note_brute_pourcentage_d'étudiants_internationaux",
        '2': "note_brute_diplômés_en_poste_à_l'international",
        '3': "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_stage",
        '4': "note_brute_diplômés_ayant_passé_plus_d'un_semestre_à_l'international_en_échange_académique",
        '5': "score_diplômés_en_poste_à_l'international"
    },
    "Formation & Effectifs": {
        '1': "note_brute_cycle_ingénieur_-_nombre_d'intégrés_en_1ère_année",
        '2': "note_brute_cycle_prépa_intégrée_-_nombre_d'intégrés_à_bac",
        '3': "note_brute_nombre_d'étudiants_par_enseignant",
        '4': "note_brute_taille_des_promos_en_cycle_ingénieur",
        '5': "note_brute_part_etudiant_hommes",
        '6': "note_brute_part_etudiant_femmes"
    },
    "Qualité & Recherche": {
        '1': "note_brute_recherche_et_enseignement",
        '2': "note_brute_masse_et_encadrement_des_doctorants",
        '3': "score_label_dd&rs",
        '4': "score_masse_et_encadrement_des_doctorants"
    }
}

# === Affichage heatmaps + statistiques ===
for theme, criteres in criteria_dict.items():
    colonnes = list(criteres.values())
    colonnes_existantes = [c for c in colonnes if c in df.columns]

    if not colonnes_existantes:
        print(f"[{theme}] Aucune donnée disponible.\n")
        continue

    data = df[colonnes_existantes]

    # Statistiques générales
    flat_values = data.values.flatten()
    flat_values = flat_values[~pd.isnull(flat_values)]

    print(f"\n📊 {theme}")
    print(f"  Moyenne   : {flat_values.mean():.3f}")
    print(f"  Médiane   : {pd.Series(flat_values).median():.3f}")
    print(f"  Écart-type: {flat_values.std():.3f}")
    print(f"  Min       : {flat_values.min():.3f}")
    print(f"  Max       : {flat_values.max():.3f}")

    # Normalisation des données pour comparabilité
    data_normalized = data.copy()
    for col in data.columns:
        max_val = df[col].max()
        if max_val != 0:
            data_normalized[col] = df[col] / max_val
        else:
            data_normalized[col] = 0

    # Heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(data_normalized, cmap='RdBu_r', linewidths=0.5, linecolor='gray',
                cbar_kws={'label': 'Score normalisé (0-1)'})
    plt.title(f"🔍 Heatmap - {theme}")
    plt.xlabel("Critères")
    plt.ylabel("Écoles")
    plt.xticks(ticks=[i + 0.5 for i in range(len(data.columns))],
               labels=[col.replace("note_brute_", "").replace("score_", "").replace("_", " ")[:25] for col in data.columns],
               rotation=30, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
from sklearn.ensemble import (
    RandomForestRegressor,
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
)
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import numpy as np

# Colonnes explicatives pertinentes
features = [
    'note_brute_insertion_à_deux_mois',
    'note_brute_diplômés_en_poste_à_l\'international',
    'note_brute_forums_entreprises',
    'note_brute_label_dd&rs',
    'note_brute_masse_et_encadrement_des_doctorants',
    'note_brute_moyenne_au_bac_des_intégrés',
    'note_brute_ouverture_sociale',
    'note_brute_part_d\'enseignants-chercheurs',
    'note_brute_politique_de_chaires',
    "note_brute_pourcentage_d'étudiants_internationaux",
    'note_brute_taux_d\'alternants',
    'note_brute_étudiants_rémunérés_pendant_leurs_études',
    'note_brute_part_etudiant_hommes',
    'note_brute_part_etudiant_femmes',
]

# Préparation des données
X = df[features].apply(pd.to_numeric, errors='coerce')
X = X.fillna(X.mean())

ecoles = df["ecole"] if "ecole" in df.columns else [f"École {i}" for i in range(len(df))]

y_proxy = df['note_brute_insertion_à_deux_mois'].fillna(
    df['note_brute_insertion_à_deux_mois'].mean()
)

# Pas de split ici puisque tu entraînes sur toutes les données

# Initialisation des modèles
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_dt = DecisionTreeRegressor(random_state=42)
model_gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Entraînement
model_rf.fit(X, y_proxy)
model_dt.fit(X, y_proxy)

model_gb = HistGradientBoostingRegressor(random_state=42)
model_gb.fit(X, y_proxy)


# Prédictions
y_pred_rf = model_rf.predict(X)
y_pred_dt = model_dt.predict(X)
y_pred_gb = model_gb.predict(X)

# Affichage des prédictions
print(
    f"{'École':30} | {'RF 6 mois':>10} | {'DT 6 mois':>10} | {'GB 6 mois':>10} | {'Insertion 2 mois (réel)':>20}"
)
print("-" * 90)
for nom_ecole, pred_rf, pred_dt, pred_gb in zip(ecoles, y_pred_rf, y_pred_dt, y_pred_gb):
    vrai_2_mois = df.loc[df['ecole'] == nom_ecole, 'note_brute_insertion_à_deux_mois'].values
    vrai_2_mois = vrai_2_mois[0] if len(vrai_2_mois) > 0 else None
    print(
        f"{nom_ecole:30} | {pred_rf:10.2f} | {pred_dt:10.2f} | {pred_gb:10.2f} | {str(vrai_2_mois):>20}"
    )

# Vraies valeurs pour correspondre aux écoles
vrais_2_mois = []
for nom_ecole in ecoles:
    val = df.loc[df['ecole'] == nom_ecole, 'note_brute_insertion_à_deux_mois'].values
    vrais_2_mois.append(val[0] if len(val) > 0 else np.nan)

x = np.arange(len(ecoles))
width = 0.25

# Calcul des différences pour chaque modèle
differences_rf = np.array(y_pred_rf) - np.array(vrais_2_mois)
differences_dt = np.array(y_pred_dt) - np.array(vrais_2_mois)
differences_gb = np.array(y_pred_gb) - np.array(vrais_2_mois)

plt.figure(figsize=(14, 6))

plt.bar(x - width, differences_rf, width, label='Différence RF', color='tab:blue', alpha=0.7)
plt.bar(x, differences_dt, width, label='Différence DT', color='tab:red', alpha=0.7)
plt.bar(x + width, differences_gb, width, label='Différence GB', color='tab:purple', alpha=0.7)

plt.axhline(0, color='black', linewidth=0.8)
plt.ylabel('Différence (Prédit - Réel)')
plt.xlabel('Écoles (non affichées)')
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# Ne pas afficher les labels x pour plus de lisibilité
plt.xticks([])

plt.tight_layout()
plt.show()